In [1]:
import pandas as pd
import pdfkit
from xlsx2html import xlsx2html

In [2]:
import re
import time
from win32com import client
import win32com.client
from pywintypes import com_error

In [3]:
import openpyxl
import PIL
from PIL import Image
from openpyxl import load_workbook
from openpyxl import Workbook

In [4]:
# gmail imports
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import base64
import mimetypes
import os
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email import encoders

In [5]:
# gui imports
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
from tkinter import Tk, Canvas, Entry, Text, Button, PhotoImage


In [6]:
# Gmail creds
SCOPES = ['https://mail.google.com/']

creds = None

if os.path.isfile('token.json'):
    os.remove('token.json')

In [7]:
date = ''

In [8]:
masterFile = r''

In [9]:
#sender = 'sell@courtorder.co.za'
sender = 'bilalc8@gmail.com'

In [10]:
finalEmails = {}

In [11]:
consignerSummary = pd.DataFrame()

In [12]:
totalMail = 0
progress = 0
failed = 0

successList = []
failList = []

In [13]:
def prep_consign_list():
    global consignerSummary
    consignerSummary = pd.read_excel(masterFile,sheet_name='Summary Per Consigner')
    print(type(consignerSummary))

    emailArrLen = len(consignerSummary)
    display(consignerSummary)
    
    
    #excel_file = masterFile
    #wb = load_workbook(excel_file, data_only = True)
    #sh = wb['Summary Per Consigner']
    #print(type(sh))
    #consignerSummary['Colour'] = ''
    #display(consignerSummary)
    #for i in range(len(consignerSummary)):

        #consignerSummary.iloc[i,consignerSummary.columns.get_loc('Colour')] = sh['A'+str(i+2)].fill.start_color.index
    ##consignerSummary['Colour'] = sh['A'].fill.start_color.index
    #global consign
    #consign = consignerSummary[consignerSummary.Colour == 'FF00FF00']
    #consign

In [14]:
def prep_mails():
    global consignerSummary
    consignerSummary = pd.read_excel(masterFile,sheet_name='Summary Per Consigner')
    print(type(consignerSummary))

    emailArrLen = len(consignerSummary)
    display(consignerSummary)
    
    
    excel_file = masterFile
    wb = load_workbook(excel_file, data_only = True)
    sh = wb['Summary Per Consigner']
    print(type(sh))
    #consignerSummary['Colour'] = ''
    display(consignerSummary)
    #for i in range(len(consignerSummary)):

        #consignerSummary.iloc[i,consignerSummary.columns.get_loc('Colour')] = sh['A'+str(i+2)].fill.start_color.index
    ##consignerSummary['Colour'] = sh['A'].fill.start_color.index

    #consign = consignerSummary[consignerSummary.Colour == 'FF00FF00']
    #consign
    # lets get number of unique email addys

    # the floats as nan cause a problem so cast the column as string
    consignerSummary["Consignor"] = consignerSummary["Consignor"].astype("str")
    dupeEmails = list(consignerSummary['Consignor'].to_numpy())

    # someone keeps leaving a newline after some emails, we must remove them to get an accurate set
    newEmails = []
    for sub in dupeEmails:
        newEmails.append(sub.replace("\n", ""))
    
    global finalEmails

    finalEmails = set(newEmails)
    #finalEmails.remove("nan")
    print(finalEmails)
    print(len(finalEmails))
    global totalMail
    totalMail = len(finalEmails)

In [15]:
#rslt_df = consign[consign['Consignor'] == 'renee81smith@gmail.com'] 
#rslt_df

In [16]:
temp_file = 'Payment Advice Template Blank.xlsx' 
wb2 = load_workbook(temp_file, data_only = True)
sh2 = wb2['Sheet2']
#copy = sh2 # we will work wth the copy from now on
#print(copy)

sheets = wb2.sheetnames # ['Sheet1', 'Sheet2']

for s in sheets:

    if s != 'Sheet2':
        sheet_name = wb2[s]
        wb2.remove(sheet_name)

# your final wb with just Sheet1



In [17]:
##################################################################################
#        ___                          __  __ 
#  ,"___".   _ _____      ___ _   LJ  LJ 
#  FJ---L]  J '_  _ `,   F __` L      FJ 
# J |  [""L | |_||_| |  | |--| |  FJ J  L
# | \___] | F L LJ J J  F L__J J J  LJ  L
# J\_____/FJ__L LJ J__LJ\____,__LJ__LJ__L
#  J_____F |__L LJ J__| J____,__F|__||__|
##################################################################################


def sendMail(email,date,htmlTable):
    global failed
    global failList
    global progress
    global successList
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().labels().list(userId='me').execute()
        labels = results.get('labels', [])

        #if not labels:
            #print('No labels found.')
            #return
        #print('Labels:')
        #for label in labels:
            #print(label['name'])
        ##########################################################################    
        mime_message = MIMEMultipart()
        
       #body
        body = f'''Good day,

We hope this email finds you well.

Please find your payment advice below.

We have already effected the payment; it should reflect in your account over the next two days.

Feel free to respond to this email with any queries that you may have.

Thanks for choosing to sell with us!

Kind Regards,

Seller Support
        '''
        # embed and hyperlink picture
        html1 = """\
        <html>
        <head></head>
        <body>
        <a href="https://courtorder.co.za/">
        <div style="display:block; text-align:center">
        <a href="https://courtorder.co.za/"> <img src="https://lh3.googleusercontent.com/Z8xy-o9VmyKN33WKBNpMG5HB2aqwFFk1hfhk0yCUomNV36jPai1oy6P3-P1ee72DR6vwDn-4zmm-L6mLWuQJvzOn3quDOFne2Ou_weryATDBZf7i9en-3CBm2kY5JHJR8PToMhUN=w596-h315-p-k" width="160 px"/> </a></div>
        </a>
        </body>
        </html>
        """
        #HTML
        html2 = """\
        <html>
        <head></head>
        <body>
        
        <div style="display:block; text-align:center">
        </div>
        <div style="display:block; text-align:center">
        <a href="https://lh3.googleusercontent.com/EslpMcpT3VVj9yh_yg9C1-FyGc7wUOUu7ED2g6sYaP6QGkfZ6IKF3Rww0Am5iczfSMXGpXaldOAplqdsFhiZGXYk28Ntzy2C-Rmnz9H5UjzQZcJTWcO3IUHFSnj7C8OIaoWTvqG_=w2400?source=screenshot.guru"> <img src="https://lh3.googleusercontent.com/EslpMcpT3VVj9yh_yg9C1-FyGc7wUOUu7ED2g6sYaP6QGkfZ6IKF3Rww0Am5iczfSMXGpXaldOAplqdsFhiZGXYk28Ntzy2C-Rmnz9H5UjzQZcJTWcO3IUHFSnj7C8OIaoWTvqG_=w320-h315-p-k" width="25" height="25"/> </a>
        <p>sell@courtorder.co.za</p>
        </div>
        <div style="display:block; text-align:center">
        <a href="https://lh3.googleusercontent.com/NVof8WV5qLq79yN_vWp_RLI8U4KLdACx1KyDeGZO_tzGZMhhzKMMavxM-Pt3vXsBcFcHWf7r5M6g7syB8Rb5TTvUrgWm9Gnm6IIjN5jN7ea0fXHJ0M22CY6Zza7AikMaLK4wNN3d=w2400?source=screenshot.guru"> <img src="https://lh3.googleusercontent.com/NVof8WV5qLq79yN_vWp_RLI8U4KLdACx1KyDeGZO_tzGZMhhzKMMavxM-Pt3vXsBcFcHWf7r5M6g7syB8Rb5TTvUrgWm9Gnm6IIjN5jN7ea0fXHJ0M22CY6Zza7AikMaLK4wNN3d=w600-h315-p-k" width="25" height="25"/> </a>
        <p>www.courtorder.co.za</p>
        </div>
        </body>
        </html>"""
        
        html3 = """\
        <html>
        <head></head>
        <body>
        <div style="display:block; text-align:center; height:20px">
        </div>
        </body>
        </html>
        """
        
        
        # headers
        mime_message['To'] = f'{email}'
        mime_message['From'] = sender
        mime_message['Subject'] = f'Court Order Payment Advice: {date}'
        
        # This example assumes the image is in the current directory
        #fp = open('https://photos.app.goo.gl/PEEaeKAxk9V2MpAWA', 'rb')
        #msgImage = MIMEImage(fp.read())
        #fp.close()
        
        # Define the image's ID as referenced above
        #msgImage.add_header('Content-ID', '<image1>')
        #mime_message.attach(msgImage)

        # text plus HTML
       
        
        mime_message.attach(MIMEText(body, 'plain'))
        mime_message.attach(MIMEText(html1, 'html'))
        mime_message.attach(MIMEText(htmlTable, 'html'))
        
        mime_message.attach(MIMEText(html3, 'html'))
       
        # attachment
       ## attachment_filename = f''
        
        # open the file in bynary
        #binary_pdf = open(attachment_filename, 'rb')
        #payload = MIMEBase('application', 'octate-stream', Name=attachment_filename)
        #payload.set_payload((binary_pdf).read())
        
        # enconding the binary into base64
        #encoders.encode_base64(payload)
        
        # add header with pdf name
        #payload.add_header('Content-Decomposition', 'attachment', filename=attachment_filename)
        #mime_message.attach(payload)
        
        
        
        # guessing the MIME type
        #type_subtype, _ = mimetypes.guess_type(attachment_filename)
        #maintype, subtype = type_subtype.split('/')

        #with open(attachment_filename, 'rb') as fp:
        #    attachment_data = fp.read()
        #mime_message.add_attachment(attachment_data, maintype, subtype)

        encoded_message = base64.urlsafe_b64encode(mime_message.as_bytes()).decode()

        create_draft_request_body = {
            'message': {
                'raw': encoded_message
           }
        }
        # pylint: disable=E1101
        draft = service.users().drafts().create(userId="me",
                                                body=create_draft_request_body)\
            .execute()
        print(F'Draft id: {draft["id"]}\nDraft message: {draft["message"]}')
    
    except HttpError as error:
        print(F'An error occurred: {error}')
        draft = None
        failed += 1
        failList.append(email)
        
    
    progress += 1
    
    successList.append(email)
    
    
    
    canvas.itemconfig(text_id, text=f"progress: {progress} / {totalMail} | {failed} failed")
    window.update()
    canvas.after(200)
    #window.update() # Update the GUI
    
    return draft
    


In [18]:
#percentage
def make_percent(x):
    return str(x * 100) + '%'

def make_rand(x):
    return 'R' + str("%0.2f" % (x,)) 
    

In [19]:
#lets create a new xlsx sheet for each person
# each payment advice should have cols
#Description
#Variant
#Code
#List Price
#Discount
#Selling Price
# % com
# comm
# payout
def send_all_mail():
    global date
    #prep_consign_list()
    prep_mails()
    print(finalEmails)
    for i in finalEmails:
        window.update()

        #print(count)

        temp_file = 'Payment Advice Template Blank.xlsx' 

        wb2c = load_workbook(temp_file, data_only = True)
        sh2c = wb2c['Sheet2']
        print(i)
        rslt_df = consignerSummary.loc[consignerSummary['Consignor'] == i] 
        df2=rslt_df.reindex(columns= ['Product Title', 
                                      'Variant Title', 
                                      'Barcode',
                                      'Price',
                                      'Discount',
                                      'Selling Price',
                                      'Commission Rate',
                                      'Commission Amount',
                                      'Payout Amount'])

        total = round(df2['Payout Amount'].sum(),2)
        print(total)
        #print(str(df2.iloc[0,0]))
        #copy = sh2
        #####################################################################################################
        #run some functions on the columns
        #####################################################################################################
        #percentage
        df2['Commission Rate'] = df2['Commission Rate'].apply(make_percent)

        #Rands sign
        df2['Price'] = df2['Price'].apply(make_rand)
        df2['Discount'] = df2['Discount'].apply(make_rand)
        df2['Selling Price'] = df2['Selling Price'].apply(make_rand)

        df2['Commission Amount'] = df2['Commission Amount'].apply(make_rand)
        df2['Payout Amount'] = df2['Payout Amount'].apply(make_rand)













        #####################################################################################################
        #CALCULATIONS
        #####################################################################################################
        count = 0
        df3 = pd.DataFrame([['','','','','','','','','']], columns= ['Product Title',
                                                                     'Variant Title', 
                                                                     'Barcode',
                                                                     'Price',
                                                                     'Discount',
                                                                     'Selling Price',
                                                                     'Commission Rate',
                                                                     'Commission Amount',
                                                                     'Payout Amount'])
        display(df3)
        df3['Payout Amount'].loc[df3.index[0]] = 'R' + str("%0.2f" % (total,))
        df3['Commission Amount'].loc[df3.index[0]] = 'Total'


        df4 = pd.concat([df2, df3])
        df4.rename(columns = {'Price':'Price(Listed)', 
                              'Selling Price':'Price(Sale)',
                              'Commission Rate':'Comm(%)',
                              'Commission Amount':'Comm',
                              'Product Title':'Product',
                              'Variant Title':'Variant',
                              'Payout Amount':'Payout'}, inplace = True)



        htmlFrame = df4.to_html(index=False)
        #replace text-align: right;
        #border: 1px solid #ddd; padding: 8px;

        htmlFrame = htmlFrame.replace("text-align: right;","padding-top: 12px; padding-bottom: 12px; text-align: center; background-color: #000000; color: white;")
        htmlFrame = htmlFrame.replace('<table border="1" class="dataframe">','<table style="font-family: Arial, Helvetica, sans-serif; border-collapse: collapse;width: 100%;">')
        htmlFrame = htmlFrame.replace('<th>','<th style="border: 1px solid #ddd; padding: 8px;">')
        htmlFrame = htmlFrame.replace('<td>','<td style="border: 1px solid #ddd; padding: 8px; text-align: center">')
        #print(htmlFrame)
        display(df4)
        #sh2c.cell(row=13, column=1).value = str(i) 
        #count = 0
        #for y in range(len(rslt_df)):
            #print(y)
            #for x in range(9):
                #try:
                #if (x==3 or x==4 or x==5 or x==7 or x==8):
                    #sh2c.cell(row=y+17, column=x+1).value = round(float(df2.iloc[y,x]),2)
                #else:
                    #sh2c.cell(row=y+17, column=x+1).value = str(df2.iloc[y,x])
            #count += round(float(str(df2.iloc[y,8])),2)
        # calculate totals
        #sh2c.cell(row=len(rslt_df)+18, column=9).value = count
        #wb2c.save('excels/test_'+i+'.xlsx')

        sendMail(i,date,htmlFrame)
        
    # some report stuff
    # create a new file
    
    global failList
    global successList

    file = open(f"report for {date}.txt", "w")
    file.close()
    
    #problem with the lists where the success showed the failed as well
    list1 = successList
    list2 = failList

    new_success_list = [item for item in list1 if item not in list2]
    
    # open the file in append mode
    with open(f"report for {date}.txt", "a") as file:
        # write the first list to the file
        file.write("failed to send:" + "\n")
        for item in failList:
            file.write(item + "\n")
        file.write("\n")
        # write the second list to the file
        file.write("successfully sent:" + "\n")
        for item in new_success_list:
            file.write(item + "\n")








In [20]:
def setDate():
    global date
    date = entry_1.get()

    

In [21]:
#def on_button_click():
    #label.config(text="Hello World!")

def on_select_file():
    global masterFile
    masterFile = filedialog.askopenfilename()
    entry_2.delete(1.0, tk.END) # Delete current text
    entry_2.insert(tk.END, masterFile) # Insert new text
    

    #label.config(text="Selected file: " + masterFile)

#root = tk.Tk()
#root.title("Court Order Automail")



#label2 = tk.Label(root, text="Select a file")
#label2.pack()

#label = tk.Label(root, text="Click the button")
#label.pack()

#button = tk.Button(root, text="Click me!", command=send_all_mail)
#button.pack()

#select_file_button = tk.Button(root, text="Select file", command=on_select_file)
#select_file_button.pack()

#window_width = 300
#window_height = 200

# get the screen dimension
#screen_width = root.winfo_screenwidth()
#screen_height = root.winfo_screenheight()

# find the center point
#center_x = int(screen_width/2 - window_width / 2)
#center_y = int(screen_height/2 - window_height / 2)

# set the position of the window to the center of the screen
#root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')

#root.mainloop()


# from tkinter import *
# Explicit imports to satisfy Flake8



#OUTPUT_PATH = Path(__file__).parent
ASSETS_PATH = Path(r"assets")
frame0 = Path(r"frame0")


def relative_to_assets(path: str) -> Path:
    return ASSETS_PATH /frame0/ Path(path)


window = Tk()

window.title("Court Order | Automail")

window.geometry("576x632")
window.configure(bg = "#FFFFFF")


canvas = Canvas(
    window,
    bg = "#FFFFFF",
    height = 632,
    width = 576,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge"
)

canvas.place(x = 0, y = 0)
canvas.create_rectangle(
    0.0,
    98.0,
    576.0,
    632.0,
    fill="#3D3D3D",
    outline="")

button_image_1 = PhotoImage(
    file=relative_to_assets("button_1.png"))
button_1 = Button(
    image=button_image_1,
    borderwidth=0,
    highlightthickness=0,
    command=lambda: [setDate(), send_all_mail()],
    relief="flat"
    )
button_1.place(
    x=332.0,
    y=494.0,
    width=214.0,
    height=49.0
)

canvas.create_rectangle(
    27.0,
    177.0,
    546.0,
    245.0,
    fill="#1E1E1E",
    outline="")

canvas.create_rectangle(
    27.0,
    274.0,
    546.0,
    396.0,
    fill="#1E1E1E",
    outline="")

canvas.create_text(
    55.0,
    198.0,
    anchor="nw",
    text="For the week ending:",
    fill="#D9D9D9",
    font=("Inter Bold", 20 * -1)
)

canvas.create_text(
    59.0,
    291.0,
    anchor="nw",
    text="2. Select Consigner file:",
    fill="#D9D9D9",
    font=("Inter Bold", 20 * -1)
)

entry_image_1 = PhotoImage(
    file=relative_to_assets("entry_1.png"))
entry_bg_1 = canvas.create_image(
    407.0,
    210.5,
    image=entry_image_1
)
entry_1 = Entry(
    bd=0,
    bg="#D9D9D9",
    fg="#000716",
    highlightthickness=0,
)
entry_1.place(
    x=310.0,
    y=190.0,
    width=194.0,
    height=39.0
)

#entry_1.bind("<FocusOut>", setDate)

image_image_1 = PhotoImage(
    file=relative_to_assets("image_1.png"))
image_1 = canvas.create_image(
    221.0,
    49.0,
    image=image_image_1
)

button_image_2 = PhotoImage(
    file=relative_to_assets("button_2.png"))
button_2 = Button(
    image=button_image_2,
    borderwidth=0,
    highlightthickness=0,
    command=on_select_file,
    relief="flat"
)
button_2.place(
    x=314.59442138671875,
    y=287.0,
    width=189.7552490234375,
    height=42.0
)

image_image_2 = PhotoImage(
    file=relative_to_assets("image_2.png"))
image_2 = canvas.create_image(
    363.0,
    49.0,
    image=image_image_2
)

image_image_3 = PhotoImage(
    file=relative_to_assets("image_3.png"))
image_3 = canvas.create_image(
    296.0,
    49.0,
    image=image_image_3
)

entry_image_2 = PhotoImage(
    file=relative_to_assets("entry_2.png"))
entry_bg_2 = canvas.create_image(
    282.0,
    362.5,
    image=entry_image_2
)
entry_2 = Text(
    bd=0,
    bg="#D9D9D9",
    fg="#000716",
    highlightthickness=0
)
entry_2.place(
    x=60.0,
    y=346.0,
    width=444.0,
    height=31.0
)

image_image_4 = PhotoImage(
    file=relative_to_assets("image_4.png"))
image_4 = canvas.create_image(
    290.0,
    141.0,
    image=image_image_4
)

text_id = canvas.create_text(
    34.0,
    450.0,
    anchor="nw",
    text=f"progress: {progress} / {totalMail} | {failed} failed",
    fill="#FFFFFF",
    font=("Inter Bold", 20 * -1)
)

image_image_5 = PhotoImage(
    file=relative_to_assets("image_5.png"))
image_5 = canvas.create_image(
    443.0,
    578.0,
    image=image_image_5
)
window.resizable(False, False)
window.mainloop()


<class 'pandas.core.frame.DataFrame'>


,Order Number,Order Date,Consignor,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount,Fulfillment Status (Line Item),Payment Gateway Names,Location
0,#57879,11-15-2022 16:13,01icedcoffee@gmail.com,Jordan 1 Low Bred Toe -uk9 - Code: I-t4un8ml92...,Uk 9 / New,I-t4un8ml92wmjiu,3499,0,3499,0.175,612.325,2886.675,fulfilled,cash,Cape Town
1,#57823,11-14-2022 22:42,aadila.eb@gmail.com,Funko POP! Mike Tyson Boxing 01,OS / New,3CChyds6E,799,0,799,0.250,199.750,599.250,unfulfilled,PayGate PayWeb,Johannesburg
2,#57889,11-15-2022 17:39,aaronzetler@gmail.com,Fear of God Essentials Sweatshorts Egg Shell,L / New,I-1v6ml8ymolky,3799,200,3599,0.225,809.770,2789.230,fulfilled,Card Payment,Cape Town
3,#57872,11-15-2022 15:43,aaronzetler@gmail.com,Fear of God Essentials Sweatshorts Smoke,L / New,I-1v6ml8ym6scs,3799,0,3799,0.225,854.770,2944.230,fulfilled,PayGate PayWeb,Johannesburg
4,#58209,11-19-2022 15:05,aaronzetler@gmail.com,Fear of God Essentials T-shirt Off Black,L / New,I-10otnhsml9wovonk,2999,0,2999,0.225,674.770,2324.230,fulfilled,Card Payment,Cape Town
5,#58202,11-19-2022 14:18,aaronzetler@gmail.com,Fear of God Essentials T-shirt Off Black,S / New,I-10otnhsml9wovogz,2999,0,2999,0.225,674.770,2324.230,fulfilled,Card Payment,Johannesburg
6,#57886,11-15-2022 17:13,abdullahggabriels@gmail.com,Jordan 1 Low Bred Toe,Uk 10 / New,I-lg6wml6kv6ik0,3799,100,3699,0.175,647.320,3051.680,fulfilled,Card Payment,Cape Town
7,#57886,11-15-2022 17:13,Gandalf,Jordan 1 Low Bred Toe,Uk 10 / New,I-lg6wml6kv6ik0,3799,100,3699,0.175,647.320,3051.680,fulfilled,Card Payment,Cape Town


<class 'openpyxl.worksheet.worksheet.Worksheet'>


,Order Number,Order Date,Consignor,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount,Fulfillment Status (Line Item),Payment Gateway Names,Location
0,#57879,11-15-2022 16:13,01icedcoffee@gmail.com,Jordan 1 Low Bred Toe -uk9 - Code: I-t4un8ml92...,Uk 9 / New,I-t4un8ml92wmjiu,3499,0,3499,0.175,612.325,2886.675,fulfilled,cash,Cape Town
1,#57823,11-14-2022 22:42,aadila.eb@gmail.com,Funko POP! Mike Tyson Boxing 01,OS / New,3CChyds6E,799,0,799,0.250,199.750,599.250,unfulfilled,PayGate PayWeb,Johannesburg
2,#57889,11-15-2022 17:39,aaronzetler@gmail.com,Fear of God Essentials Sweatshorts Egg Shell,L / New,I-1v6ml8ymolky,3799,200,3599,0.225,809.770,2789.230,fulfilled,Card Payment,Cape Town
3,#57872,11-15-2022 15:43,aaronzetler@gmail.com,Fear of God Essentials Sweatshorts Smoke,L / New,I-1v6ml8ym6scs,3799,0,3799,0.225,854.770,2944.230,fulfilled,PayGate PayWeb,Johannesburg
4,#58209,11-19-2022 15:05,aaronzetler@gmail.com,Fear of God Essentials T-shirt Off Black,L / New,I-10otnhsml9wovonk,2999,0,2999,0.225,674.770,2324.230,fulfilled,Card Payment,Cape Town
5,#58202,11-19-2022 14:18,aaronzetler@gmail.com,Fear of God Essentials T-shirt Off Black,S / New,I-10otnhsml9wovogz,2999,0,2999,0.225,674.770,2324.230,fulfilled,Card Payment,Johannesburg
6,#57886,11-15-2022 17:13,abdullahggabriels@gmail.com,Jordan 1 Low Bred Toe,Uk 10 / New,I-lg6wml6kv6ik0,3799,100,3699,0.175,647.320,3051.680,fulfilled,Card Payment,Cape Town
7,#57886,11-15-2022 17:13,Gandalf,Jordan 1 Low Bred Toe,Uk 10 / New,I-lg6wml6kv6ik0,3799,100,3699,0.175,647.320,3051.680,fulfilled,Card Payment,Cape Town


{'aadila.eb@gmail.com', 'Gandalf', '01icedcoffee@gmail.com', 'aaronzetler@gmail.com', 'abdullahggabriels@gmail.com'}
5
{'aadila.eb@gmail.com', 'Gandalf', '01icedcoffee@gmail.com', 'aaronzetler@gmail.com', 'abdullahggabriels@gmail.com'}
aadila.eb@gmail.com
599.25


...

,Product,Variant,Barcode,Price(Listed),Discount,Price(Sale),Comm(%),Comm,Payout
2,Fear of God Essentials Sweatshorts Egg Shell,L / New,I-1v6ml8ymolky,R3799.00,R200.00,R3599.00,22.5%,R809.77,R2789.23
3,Fear of God Essentials Sweatshorts Smoke,L / New,I-1v6ml8ym6scs,R3799.00,R0.00,R3799.00,22.5%,R854.77,R2944.23
4,Fear of God Essentials T-shirt Off Black,L / New,I-10otnhsml9wovonk,R2999.00,R0.00,R2999.00,22.5%,R674.77,R2324.23
5,Fear of God Essentials T-shirt Off Black,S / New,I-10otnhsml9wovogz,R2999.00,R0.00,R2999.00,22.5%,R674.77,R2324.23
0,,,,,,,,Total,R10381.92


Draft id: r-2491009166375072761
Draft message: {'id': '185bf78154eb74df', 'threadId': '185bf78154eb74df', 'labelIds': ['DRAFT']}
abdullahggabriels@gmail.com
3051.68


,Product Title,Variant Title,Barcode,Price,Discount,Selling Price,Commission Rate,Commission Amount,Payout Amount
0,,,,,,,,,


,Product,Variant,Barcode,Price(Listed),Discount,Price(Sale),Comm(%),Comm,Payout
6,Jordan 1 Low Bred Toe,Uk 10 / New,I-lg6wml6kv6ik0,R3799.00,R100.00,R3699.00,17.5%,R647.32,R3051.68
0,,,,,,,,Total,R3051.68


Draft id: r-3678956340047093073
Draft message: {'id': '185bf7818ef869de', 'threadId': '185bf7818ef869de', 'labelIds': ['DRAFT']}
